**Dashboard proyecto ingenieria BOSTON**

In [ ]:
%%writefile app.py

import streamlit as st
import plotly.express as px
import pandas as pd
from streamlit_option_menu import option_menu #Añadir más interactividad con el usuario con un menú
from streamlit_calendar import calendar #Agregar calendario (pendiente)
import os

# AGREGAR LA PARTE DE TIPOS DE GRÁFICAS (Regresión Lineal, Múltiple, Análisis Univariado, Extracción de características
# y Regresión Logística) Mejorar el diseño, añadir mapa de calor (heatmap), añadir la parte en donde se muestran los puntos
# focales que van en color rojo.(De la parte de gráfica de dispersión, aunque creo que ya está implementado por la parte de
# que se hizo una opción de ir quitando la parte de room_type)

def load_css(file_name):
    st.markdown("""
        <link href="https://fonts.googleapis.com/css2?family=Montserrat:ital,wght@0,100..900;1,100..900&display=swap" rel="stylesheet">
    """, unsafe_allow_html=True)
    with open(file_name) as f:
        st.markdown(f"<style>{f.read()}</style>", unsafe_allow_html=True)

load_css("css/style.css") #Uso de css (carga)

@st.cache_resource
def load_data():
    df = pd.read_csv("data/Boston_Limpio_Procesado.csv")

    numeric_df = df.select_dtypes(include=['float', 'int'])
    numeric_cols = numeric_df.columns

    text_df = df.select_dtypes(include=['object'])
    text_cols = text_df.columns

    categorical_column = df['room_type']
    unique_categories = categorical_column.unique()

    return df, numeric_cols, text_cols, unique_categories, numeric_df

df, numeric_cols, text_cols, unique_categories, numeric_df = load_data()

with st.sidebar:
    View = option_menu(
        menu_title="Menú Principal",
        options=[
                    "Inicio", 
                    "Análisis Temporal", 
                     "Precios y Ocupación", 
                     "Disponibilidad y Ubicación", 
                     "Comparativa General (Ciudades)", 
                     "Galería Hospedajes Airbnb",
                     ], 
        icons=[
                "house", 
                "clock", 
                "bar-chart", 
                "calendar", 
                "list-task", 
                "journal-album",
                ],
        menu_icon="cast",
        default_index=0,
        styles={
            "container": {"background-color": "#86A788"},
            "icon": {"color": "#F1E7E7", "font-size": "20px"},
            "nav-link": {
                "font-size": "16px",
                "text-align": "left",
                "margin": "0px",
                "--hover-color": "#FF9A9A",
            },
            "nav-link-selected": {
                "background-color": "#FF9A9A",
                "color": "white",
            },
        }
    )
   
#Analizar variables numéricas por tipo de habitación (ver tendencias de precios, disponbilidad, num de reseñas)

if View == "Inicio":
    st.title("Análisis comparativo de Airbnb")
    
    st.image("img/Boston.jpeg", use_container_width=True)
    st.subheader("Boston, Massachusets")

    st.write("""
        Fundada en 1630, una de las ciudades más antiguas y emblemáticas de Estados Unidos.
        Conocida por su papel fundamental en la historia americana, su riqueza cultural, y su vibrante 
        escena académica, Boston combina tradición y modernidad de manera única.

        Este dashboard presenta un análisis detallado sobre los datos de Airbnb Boston, permitiendo explorar
        variables clave como precios, tipos de habitación, disponibilidad, entre otros.

        Nuestro objetivo es fortalecer una visión clara y comparativa que permita comprender mejor el mercado
        en esta ciudad histórica y dinámica. 
    """)

if View == "Análisis Temporal":
    st.sidebar.header("Análisis Temporal")
    #st.sidebar.subheader("Gráfica de línea")
    #Crear otra vista para poder crear histogramas o graficos de barras por variable. (proce, availability)
    #(Regresión lineal y múltiple)

    st.sidebar.markdown("Opciones de Análisis")
    check_box = st.sidebar.checkbox(label="Mostrar Dataset")
    if check_box:
        st.write(df)
        st.write(df.describe())

    temporal_vars = ['number_of_reviews', 'availability_365', 'reviews_per_month', 'minimum_nights', 'host_response_rate', 'availability_90']
    temporal_vars = [var for var in temporal_vars if var in df.columns]

    numerics_vars_selected = st.sidebar.multiselect(
        label="Variables a Graficar",
        options=temporal_vars
    )

    category_selected = st.sidebar.selectbox(label="Categorías (room type)", options=unique_categories)

    data = df[df['room_type'] == category_selected]
    data_features = data[numerics_vars_selected]

    figure1 = px.line(data_frame=data_features, x=data_features.index, y=numerics_vars_selected, title="Line Plot", width=1600, height=600)

    Button2 = st.sidebar.button(label="Mostrar Gráfica Line Plot")
    if Button2:
        st.plotly_chart(figure1)

elif View == "Precios y Ocupación":

    #Para crear una mejor visualización de las correlaciones, podría crear un mapa de calor (heatmap).

    precios_ocupacion_vars = [
        'availability_365',
        'number_of_reviews',
        'reviews_per_month',
        'minimum_nights',
        'price',
        'bedrooms',
        'accommodates',
        'bathrooms'
    ]
    
    precios_ocupacion_vars = [var for var in precios_ocupacion_vars if var in df.columns]

    x_selected = st.sidebar.selectbox(label="Eje X", options=precios_ocupacion_vars)
    y_selected = st.sidebar.selectbox(label="Eje Y", options=precios_ocupacion_vars)

    figure2 = px.scatter(
        data_frame=df,
        x=x_selected,
        y=y_selected,
        color='room_type',
        title="Relación entre Variables de Precio y Ocupación"
    )

    st.plotly_chart(figure2)

elif View == "Disponibilidad y Ubicación":

    categorical_options = ['room_type', 'property_type']
    numerical_options = ['availability_365', 'price', 'number_of_reviews']

    Variable_cat = st.sidebar.selectbox(label="Variable Categórica", options=categorical_options)
    Variable_num = st.sidebar.selectbox(label="Variable Numérica", options=numerical_options)

    # Agrupar y filtrar
    df_grouped = df.groupby(Variable_cat)[Variable_num].sum().reset_index()
    df_grouped = df_grouped.sort_values(by=Variable_num, ascending=False).head(10)

    # Gráfico
    figure3 = px.pie(
        data_frame=df_grouped,
        names=Variable_cat,
        values=Variable_num,
        title=f"Distribución de {Variable_num} por {Variable_cat}",
        width=1600,
        height=600
    )
    st.plotly_chart(figure3)

elif View == "Comparativa General (Ciudades)":

    df_mexico = pd.read_csv("data/Mexico_Datos_Limpios_2.csv")
    df_boston = df

    st.sidebar.markdown("Comparación entre Boston y Ciudad de México")

    text_cols_comunes = list(set(df_boston.select_dtypes(include='object').columns) & set(df_mexico.select_dtypes(include='object').columns))
    numeric_cols_comunes = list(set(df_boston.select_dtypes(include=['float', 'int']).columns) & set(df_mexico.select_dtypes(include=['float', 'int']).columns))

    Variable_cat = st.sidebar.selectbox(label="Variable Categórica", options=text_cols_comunes)
    Variable_num = st.sidebar.selectbox(label="Variable Numérica", options=numeric_cols_comunes)

    boston_grouped = df_boston.groupby(Variable_cat)[Variable_num].mean().reset_index()
    mexico_grouped = df_mexico.groupby(Variable_cat)[Variable_num].mean().reset_index()

    col1, col2 = st.columns(2)

    with col1:
        st.markdown("Boston")
        fig_boston = px.bar(boston_grouped, x=Variable_cat, y=Variable_num, title=f"{Variable_num} - {Variable_cat}")
        st.plotly_chart(fig_boston, use_container_width=True)

    with col2:
        st.markdown("Ciudad de México")
        fig_mexico = px.bar(mexico_grouped, x=Variable_cat, y=Variable_num, title=f"{Variable_num} - {Variable_cat}")
        st.plotly_chart(fig_mexico, use_container_width=True)

elif View == "Galería Hospedajes Airbnb":
    import os
    import base64
    st.title("Galería de Hospedajes Airbnb en Boston")
    #image_folder = "img/galeria"

    imagenes = [
        {
            "archivo": "img/galeria/foto1.jpg", 
            "url": "https://www.airbnb.mx/rooms/1308821660372582773?adults=1&search_mode=regular_search&check_in=2025-05-01&check_out=2025-05-06&children=0&infants=0&pets=0&source_impression_id=p3_1745988825_P3D5tqfLeqn7tpah&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981",
            "caption": "Habitación en Hotel en Cambridge"
        },
        {
            "archivo": "img/galeria/foto2.jpg",
            "url": "https://www.airbnb.mx/rooms/1385582422054701956?adults=1&search_mode=regular_search&category_tag=Tag%3A8678&check_in=2025-06-07&check_out=2025-06-12&children=0&infants=0&pets=0&photo_id=2118105755&source_impression_id=p3_1745988826_P3d1PSxI5cTyDrAh&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981&modal=PHOTO_TOUR_SCROLLABLE",
            "caption": "Habitación en Malden"
        },
        {
            "archivo": "img/galeria/foto3.jpg",
            "url": "https://www.airbnb.mx/rooms/1388674780017597333?adults=1&search_mode=regular_search&category_tag=Tag%3A8678&check_in=2025-06-09&check_out=2025-06-14&children=0&infants=0&pets=0&photo_id=2122523176&source_impression_id=p3_1745988826_P3jVDuolqmnE6h5S&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981",
            "caption": "Habitación en Chelsea"
        },
        {
            "archivo": "img/galeria/foto4.jpg",
            "url": "https://www.airbnb.mx/rooms/24944930?adults=1&search_mode=regular_search&category_tag=Tag%3A8678&check_in=2025-08-12&check_out=2025-08-17&children=0&infants=0&pets=0&photo_id=1734219622&source_impression_id=p3_1745988826_P3g5nayK75Qs_ZWQ&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981",
            "caption": "Habitación en Boston, Massa"
        },
        {
            "archivo": "img/galeria/foto5.jpg",
            "url": "https://www.airbnb.mx/rooms/938898143869019261?adults=1&search_mode=regular_search&check_in=2025-05-04&check_out=2025-05-09&children=0&infants=0&pets=0&source_impression_id=p3_1745988826_P3zM8zumlvbQLJB-&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981&modal=PHOTO_TOUR_SCROLLABLE",
            "caption": "Habitación en hotel Chelsea"
        },
        {
            "archivo": "img/galeria/foto6.jpg",
            "url": "https://www.airbnb.mx/rooms/9282334?adults=1&search_mode=regular_search&category_tag=Tag%3A8678&check_in=2025-08-28&check_out=2025-09-02&children=0&infants=0&pets=0&photo_id=1349861290&source_impression_id=p3_1745988826_P3myVGx0ILxLJbSM&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981",
            "caption": "Habitación en Chelsea en The Washburn"
        },
        {
            "archivo": "img/galeria/foto7.jpg",
            "url": "https://www.airbnb.mx/rooms/1136902852796712722?adults=1&search_mode=regular_search&check_in=2025-05-31&check_out=2025-06-05&children=0&infants=0&pets=0&source_impression_id=p3_1745988826_P3FN7WTGFBp6tRpi&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981",
            "caption": "Habitación en The Verb Hotel"
        },
        {
            "archivo": "img/galeria/foto8.jpg",
            "url": "https://www.airbnb.mx/rooms/27908078?adults=1&search_mode=regular_search&category_tag=Tag%3A8678&check_in=2025-05-04&check_out=2025-05-09&children=0&infants=0&pets=0&photo_id=566726014&source_impression_id=p3_1745988826_P3lVEy0FahB-8fz5&previous_page_section_name=1000&federated_search_id=6028635f-b600-4a08-b8ba-286b776f8981",
            "caption": "Habitación en Medford"
        }
    ]   

    # Mostrar en columnas
    for i in range(0, len(imagenes), 2):
        cols = st.columns(2)  # 2 columnas por fila
        for j, col in enumerate(cols):
            if i + j < len(imagenes):
                item = imagenes[i + j]
                if os.path.exists(item["archivo"]):
                    with open(item["archivo"], "rb") as f:
                        img_bytes = f.read()
                    encoded = base64.b64encode(img_bytes).decode()
                    col.markdown(
                        f'''
                        <a href="{item["url"]}" target="_blank">
                            <img src="data:image/jpeg;base64,{encoded}" 
                            style="width:100%; border-radius:10px;" />
                        </a>
                        <p style="text-align:center; font-size:14px;">{item["caption"]}</p>
                        ''',
                        unsafe_allow_html=True
                    )
                else:
                    col.warning(f"No se encontró: {item['archivo']}")

Overwriting app.py
